In [1]:
from pyspark.sql import SparkSession, functions as F

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "8g")
    .config("spark.driver.memory", "8g")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/09/24 21:35:42 WARN Utils: Your hostname, LAPTOP-MMSL4U2D resolves to a loopback address: 127.0.1.1; using 172.23.193.19 instead (on interface eth0)
24/09/24 21:35:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/24 21:35:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/24 21:35:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
trans_final_sdf = spark.read.parquet('../data/merged/merged_transactions_with_outliers.parquet')
trans_final_sdf.show(5)

+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+-----------+-----+--------+------+--------------------------+--------------------------+--------------+--------------------+------------------+------------------+----------------------+--------------------+------------+--------------------+--------------------------+--------------------+--------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|consumer_id|state|postcode|gender|consumer_fraud_probability|merchant_fraud_probability| LOCALITY_NAME|       list_SA2_CODE|          list_ERP|  list_num_earners|list_total_income_($M)|    list_mean_income|postcode_ERP|postcode_num_earners|postcode_total_income_($M)|postcode_mean_income|is_fraud|
+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+-----------+-----+--------+------+-------------

In [5]:
# Function to convert to string of list
import re
def process_string(input_string):
    input_string = input_string.replace('(', '[')
    input_string = input_string.replace(')', ']')

    # Use regex to split by outer brackets and commas
    lists = re.findall(r'\[([^\[\]]+)\]', input_string)

    # Further split elements within the lists based on commas, while stripping whitespace
    result = [list(map(str.strip, item.split(','))) for item in lists]

    # Clean up each item by stripping leading/trailing whitespace and correcting case
    for sublist in result:
        for i in range(len(sublist)):
            sublist[i] = sublist[i].strip().lower()

    return result
# Register UDF in PySpark
process_string_udf = F.udf(process_string)

# Apply the UDF to the DataFrame
result_df = trans_final_sdf.withColumn("fixed_tags", process_string_udf(F.col("tags")))

# Show the result
result_df.show(truncate=False)

+-------+------------+------------------+------------------------------------+--------------+-------------------------------+-----------------------------------------------------------------------------------------------------------------+-----------+-----+--------+-----------+--------------------------+--------------------------+--------------+------------------------------------------------------------------+---------------------------------------+------------------------------------+------------------------------------------+------------------------------------------------------+------------+--------------------+--------------------------+--------------------+--------+-----------------------------------------------------------------------------------------------------------------+
|user_id|merchant_abn|dollar_value      |order_id                            |order_datetime|merchant_name                  |tags                                                                            

In [9]:
result_df.write.mode("overwrite").parquet("../data/merged/merged_transactions_with_tags.parquet")